In [1]:
import numpy as np
import sympy.utilities.iterables as itt
import itertools as it
import pandas as pd
import time

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days


In [2]:
def aprioriGen(L, k):
    l = len(L);
    Lk = [];
    for i in range(l - 1):
        for j in range(i + 1, l):
            if (np.all(L[i][:(k-1)] == L[j][:(k-1)])):
                tmp = list(L[i][:k]);
                tmp.append(L[j][k-1]);
                zgodne = 0;
                while(zgodne <= k):
                    if (np.any(np.all(np.equal(L, np.concatenate((tmp[:zgodne], tmp[(zgodne + 1):]))), axis = 1))):
                        zgodne += 1;
                    else:
                        break;
                if (zgodne == k + 1):
                    Lk.append(tmp);
            else:
                break;
    return np.array(Lk)

def convert(data):
    freq = [];
    dict1 = {};
    dict2 = [];
    dict3 = {}
    for i in range(len(data)):
        for j in range(len(data[i])):
            item = data[i][j]
            if (dict1.get(item, -1) == -1):
                dict1[item] = len(freq);
                dict2.append(item);
                freq.append(1);
                data[i][j] = dict1[item];
            else:
                data[i][j] = dict1[item];
                freq[int(dict1[item])] += 1;
    for key in dict1.keys():
        dict3[dict1[key]] = key
    return (np.array(freq).astype(float) / len(data)), dict3, dict2;


def apriori(C1, data, minsup):
    t = time.time()
    length = len(data);
    L = np.argwhere(C1 > minsup);
    k = 1;
    L = aprioriGen(L, k);
    k += 1;
    res = [];
    while (len(L) > 0):
        L1 = []
        for set1 in L:
            suma = 0.
            for transaction in data:
                if (set(set1) <= transaction):
                    suma +=1;
            sup = suma / length;
            if (sup > minsup):
                L1.append(set1);
                res.append([set1, sup]);
        L = aprioriGen(L1, k);
        k += 1;
    print 'Apriori zajelo ',time.time() - t, 'sekund'
    return res;


def confidence(data, sets, minconf):
    t = time.time()
    res = [];
    for row in sets:
        where = len(res);
        tabpom = [];
        sup = row[1];
        tmp = list(itt.subsets(row[0]));
        rozbicie = tmp[1:(len(tmp) - 1)];
        dl = len(rozbicie);
        for i in range(dl):
            suma = 0.;
            for transaction in data:   
                if( set(rozbicie[i]) <= transaction ):
                    suma += 1;
            sup2 = suma / len(data);
            tabpom.append(sup2);
            if (sup2 > 0):
                conf = sup / sup2;
                if (conf > minconf):
                    res.append([sup, sup2, dl-i-1, 0, 0, conf, list(rozbicie[i]), list(rozbicie[dl-i-1])]);
        for i in range(where, len(res)):
            res[i][2] = tabpom[res[i][2]];
    for row in res:
        row[3] = row[0] - row[1] * row[2];
        row[4] = row[5] / row[2];
    print 'Confidence zajelo',time.time() - t, 'sekund'
    return res;

labels = ['Support', 'Leverage', 'Lift', 'Confidence', 'Set 1', 'Set 2']

def write(W,dict1 = []):
    print labels
    if( len(dict1) == 0):
        for row in W:
            print "Su: %.3f" % row[0], " Le: %.3f" % row[3], " Li: %.3f" % row[4], " Co: %.3f" % row[5], row[6], "=>", row[7];
    else:
        for row in W:
            print "Su: %.3f" % row[0], " Le: %.3f" % row[3], " Li: %.3f" % row[4], " Co: %.3f" % row[5], fromDict(row[6],dict1), "=>", fromDict(row[7],dict1);
def fromDict(tab,dict1):
    tabpom = []
    for elem in tab:
        tabpom.append(dict1[elem])
    return tabpom

In [3]:
datas = 'retail.dat';
with open(datas, 'r') as f:
    retail = f.readlines()
data = np.array([np.array([int(i) for i in wiersz.split()]) for wiersz in retail])

In [4]:
C1, dict1, name = convert(data)
for i in range(len(data)):
    data[i] = set(data[i]);

In [5]:
W = apriori(C1,data,0.01)
w = confidence(data,W,0.5)
write(w,dict1)

Apriori zajelo  567.992383003 sekund
Confidence zajelo 24.5477781296 sekund
['Support', 'Leverage', 'Lift', 'Confidence', 'Set 1', 'Set 2']
Su: 0.096  Le: -0.003  Li: 0.970  Co: 0.557 [32] => [39]
Su: 0.091  Le: 0.009  Li: 1.108  Co: 0.530 [32] => [48]
Su: 0.032  Le: 0.026  Li: 5.372  Co: 0.950 [36] => [38]
Su: 0.023  Le: 0.004  Li: 1.207  Co: 0.694 [36] => [39]
Su: 0.012  Le: 0.010  Li: 5.505  Co: 0.974 [37] => [38]
Su: 0.117  Le: 0.016  Li: 1.154  Co: 0.663 [38] => [39]
Su: 0.090  Le: 0.006  Li: 1.066  Co: 0.509 [38] => [48]
Su: 0.031  Le: 0.025  Li: 5.513  Co: 0.975 [110] => [38]
Su: 0.034  Le: 0.028  Li: 5.529  Co: 0.978 [170] => [38]
Su: 0.013  Le: 0.010  Li: 5.333  Co: 0.943 [286] => [38]
Su: 0.129  Le: 0.032  Li: 1.329  Co: 0.764 [41] => [39]
Su: 0.331  Le: 0.056  Li: 1.203  Co: 0.575 [39] => [48]
Su: 0.331  Le: 0.056  Li: 1.203  Co: 0.692 [48] => [39]
Su: 0.011  Le: 0.001  Li: 1.149  Co: 0.660 [60] => [39]
Su: 0.032  Le: 0.002  Li: 1.084  Co: 0.623 [65] => [39]
Su: 0.013  Le: 0

In [6]:
w = confidence(data,W,0.74)
write(w,dict1)

Confidence zajelo 11.1217820644 sekund
['Support', 'Leverage', 'Lift', 'Confidence', 'Set 1', 'Set 2']
Su: 0.032  Le: 0.026  Li: 5.372  Co: 0.950 [36] => [38]
Su: 0.012  Le: 0.010  Li: 5.505  Co: 0.974 [37] => [38]
Su: 0.031  Le: 0.025  Li: 5.513  Co: 0.975 [110] => [38]
Su: 0.034  Le: 0.028  Li: 5.529  Co: 0.978 [170] => [38]
Su: 0.013  Le: 0.010  Li: 5.333  Co: 0.943 [286] => [38]
Su: 0.129  Le: 0.032  Li: 1.329  Co: 0.764 [41] => [39]
Su: 0.015  Le: 0.003  Li: 1.306  Co: 0.750 [2238] => [39]
Su: 0.022  Le: 0.018  Li: 5.398  Co: 0.955 [36, 39] => [38]
Su: 0.015  Le: 0.013  Li: 5.429  Co: 0.960 [36, 48] => [38]
Su: 0.013  Le: 0.003  Li: 1.371  Co: 0.788 [36, 48] => [39]
Su: 0.035  Le: 0.009  Li: 1.362  Co: 0.783 [38, 41] => [39]
Su: 0.069  Le: 0.017  Li: 1.336  Co: 0.768 [38, 48] => [39]
Su: 0.020  Le: 0.016  Li: 5.592  Co: 0.989 [39, 110] => [38]
Su: 0.023  Le: 0.019  Li: 5.543  Co: 0.981 [39, 170] => [38]
Su: 0.015  Le: 0.013  Li: 5.575  Co: 0.986 [48, 110] => [38]
Su: 0.017  Le: 0.

In [7]:
W = apriori(C1,data,0.005)
w = confidence(data,W,0.8)
write(w,dict1)

Apriori zajelo  3645.20387483 sekund
Confidence zajelo 40.5278880596 sekund
['Support', 'Leverage', 'Lift', 'Confidence', 'Set 1', 'Set 2']
Su: 0.032  Le: 0.026  Li: 5.372  Co: 0.950 [36] => [38]
Su: 0.012  Le: 0.010  Li: 5.505  Co: 0.974 [37] => [38]
Su: 0.007  Le: 0.006  Li: 5.275  Co: 0.933 [55] => [38]
Su: 0.006  Le: 0.005  Li: 5.431  Co: 0.961 [56] => [38]
Su: 0.007  Le: 0.006  Li: 5.532  Co: 0.979 [105] => [38]
Su: 0.031  Le: 0.025  Li: 5.513  Co: 0.975 [110] => [38]
Su: 0.034  Le: 0.028  Li: 5.529  Co: 0.978 [170] => [38]
Su: 0.013  Le: 0.010  Li: 5.333  Co: 0.943 [286] => [38]
Su: 0.009  Le: 0.007  Li: 5.544  Co: 0.981 [371] => [38]
Su: 0.006  Le: 0.005  Li: 5.491  Co: 0.971 [790] => [38]
Su: 0.006  Le: 0.003  Li: 1.697  Co: 0.811 [1135] => [48]
Su: 0.009  Le: 0.004  Li: 1.803  Co: 0.862 [2958] => [48]
Su: 0.007  Le: 0.007  Li: 65.190  Co: 0.973 [16011] => [16010]
Su: 0.005  Le: 0.004  Li: 5.401  Co: 0.955 [32, 36] => [38]
Su: 0.005  Le: 0.004  Li: 5.577  Co: 0.987 [32, 110] =>